##### greencar

In [ ]:
import requests
import pandas as pd
import json
import urllib

from google.oauth2 import service_account
from google.cloud import bigquery
from pandas_gbq import to_gbq

In [ ]:
address_list = [' ']
columns = ["zoneName", "address", "latitude", "longitude","type"]
api_key = "OniyFz9aRuv6SpCF+x2ytMHyEeMTJ9DXHyWdz+JL6RnGVrrEXR+aeKYZ0+4xah26a3Kuk0TTspLZ7MACxZObgg=="
url = 'http://apis.data.go.kr/1613000/CarSharingInfoService/getCarZoneListByName'

df1_1 = pd.DataFrame()

for address in address_list:
    params ={'serviceKey' : api_key, 'pageNo' : '1', 'numOfRows' : '5000', '_type' : 'json', 'zoneName' : address }
    response = requests.get(url, params=params)
    result=json.loads(response.content.decode('utf-8'))

    if "response" in result and "body" in result["response"] and "items" in result["response"]["body"] and "item" in result["response"]["body"]["items"]:
        item = result["response"]["body"]["items"]["item"]
        if type(item) == dict :
            item = [item]
            temp_df = pd.DataFrame(item)
        elif type(item) == list :
            temp_df = pd.DataFrame(item)

        df1_1 = pd.concat([df1_1, temp_df], ignore_index=True)
    else :
        continue  
df1_1 = df1_1[columns]
df1_1 = df1_1[df1_1["type"]==2]
# 운행종료 제거
df1_1_filtered = df1_1[~df1_1["zoneName"].str.contains(r"운영종료", na=False)].drop_duplicates()

#두 번째 데이터셋
df1_2 = pd.read_csv("./전국렌터카업체정보표준데이터.csv", encoding="euc-kr")
def choose_address(row):
    # 둘 다 존재하면 도로명 주소 사용
    if pd.notna(row["차고지도로명주소"]) and pd.notna(row["차고지지번주소"]):
        return row["차고지지번주소"]
    # 하나만 존재하면 해당 주소 사용
    elif pd.notna(row["차고지도로명주소"]):
        return row["차고지도로명주소"]
    elif pd.notna(row["차고지지번주소"]):
        return row["차고지지번주소"]
    # 둘 다 없으면 NaN
    else:
        return None

# 각 행에 대해 choose_address 함수 적용
df1_2["address"] = df1_2.apply(choose_address, axis=1)

columns = ['제공기관명', "address", '위도', '경도']
df1_2_filtered = df1_2[df1_2["업체명"].str.contains(r"그린카", na=False)]
df1_2_filtered = df1_2_filtered[columns]
df1_2_filtered.rename(columns={"제공기관명":"zoneName",
                              "위도":"latitude",
                              "경도":"longitude",
                              }, inplace=True)

df1_greencar = pd.concat([df1_1_filtered, df1_2_filtered], ignore_index=True)
df1_greencar["latitude"] = df1_greencar["latitude"].round(3)
df1_greencar["longitude"] = df1_greencar["longitude"].round(3)
df1_greencar.drop(columns = ["type"], inplace=True)

# 위도, 경도 기준으로 중복 제거
df1_greencar = df1_greencar.drop_duplicates(subset=["latitude", "longitude"]).reset_index(drop=True)

##### licenses

In [ ]:
df2_licenses = pd.read_excel("./운전면허소지자현황_지역별__20241208031332.xlsx")
df2_licenses = df2_licenses.iloc[-4:, 4:]
# 1행부터 4행까지 합산 (숫자 데이터만)
df2_licenses = df2_licenses.iloc[0:4, :].apply(pd.to_numeric, errors='coerce').sum(axis=0).reset_index()
df2_licenses.columns = ['Region', 'license']

# 경기남부와 경기북부 합산
gyeonggi_sum = df2_licenses.loc[df2_licenses['Region'].isin(['경기남부', '경기북부']), 'license'].sum()
# 새로운 경기 행 생성
gyeonggi_row = pd.DataFrame({'Region': ['경기'], 'license': [gyeonggi_sum]})
# 경기남부와 경기북부 제거 후 새로운 행 추가
df2_licenses = pd.concat([df2_licenses[~df2_licenses['Region'].isin(['경기남부', '경기북부'])], gyeonggi_row], ignore_index=True)

region_mapping = {
    '서울': '서울특별시',
    '세종': '세종특별자치시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '울산': '울산광역시',
    '경기': '경기도',
    '강원': '강원도',
    '경남': '경상남도',
    '경북': '경상북도',
    '전남': '전라남도',
    '전북': '전라북도',
    '충남': '충청남도',
    '충북': '충청북도',
    '제주': '제주특별자치도'
}

# Region 열 값 변경
df2_licenses['Region'] = df2_licenses['Region'].replace(region_mapping)
df2_licenses


##### transports

In [ ]:
df3_transport1 = pd.read_csv("./KC_496_LLR_PPLTEQP_2023.csv")
df3_transport1["longitude"] = df3_transport1["LC_LO"].round(3)
df3_transport1["latitude"] = df3_transport1["LC_LA"].round(3)
columns_transport = [
           "POI_NM",
           "CTPRVN_NM",
           "SIGNGU_NM",
           "LEGALDONG_NM",
           "LI_NM",
           "longitude",
           "latitude",
           ]
df3_transport1 = df3_transport1[columns_transport]
df3_transport1.fillna(0, inplace=True)
names = ["택시", "선착장", "역", "항", "여객", "유람선", "나루", "부두", "교", "댐", "다리", "광사"]
pattern = '|'.join(names)
# 불필요 제거
df3_transport = df3_transport1[~df3_transport1["POI_NM"].str.contains(pattern, na=False)].drop_duplicates()

##### sites

In [ ]:
def clean_address(address):
    # address 끝에 '-0'이 있으면 제거
    if address.endswith("-0"):
        return address[:-2]
    return address

def get_coordinates_without_api(address):
    try:
        # OpenStreetMap Nominatim 검색 URL
        base_url = "https://nominatim.openstreetmap.org/search"

        # 파라미터 구성
        params = {
            "q": address,
            "format": "json"
        }

        # 요청 URL 생성
        url = f"{base_url}?{urllib.parse.urlencode(params)}"

        # 요청 보내기
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

        # 응답 JSON 파싱
        if response.status_code == 200:
            results = response.json()
            if results:
                latitude = results[0]["lat"]
                longitude = results[0]["lon"]
                return latitude, longitude
            else:
                print("주소를 찾을 수 없습니다.")
                return None, None
        else:
            print(f"Error: HTTP {response.status_code}")
            return None, None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [ ]:
# CSV 파일 읽기
df4_poi = pd.read_csv("./20241212192423_지역별 관광지 검색순위.csv", encoding="euc-kr")
columns_poi = ['관광지명', '도로명주소', '중분류 카테고리', '소분류 카테고리','검색건수']

df4_poi = df4_poi[df4_poi['소분류 카테고리'] != "교통시설"][columns_poi].sort_values(by="검색건수", ascending=False)
df4_poi = df4_poi.rename(columns={
    '관광지명':'POI_NM',
    '도로명주소':'address',
    '중분류 카테고리':'middle category',
    '소분류 카테고리':'small category',
    '검색건수':'SCCNT_VALUE'
})

# clean_address 함수를 벡터화 연산으로 적용
df4_poi['address'] = df4_poi['address'].apply(clean_address)

# 위도와 경도를 저장할 컬럼 추가
df4_poi["latitude"] = 0
df4_poi["longitude"] = 0


# 각 address에 대해 위도, 경도를 검색하고 삽입
for index, row in df4_poi.iterrows():
    latitude, longitude = get_coordinates_without_api(row["address"])
    if latitude and longitude:
        df4_poi.at[index, "latitude"] = latitude
        df4_poi.at[index, "longitude"] = longitude

df4_poi = df4_poi[~((df4_poi['latitude'] == 0) & (df4_poi['longitude'] == 0))]

In [ ]:
df4_poi = df4_poi[~((df4_poi['latitude'] == 0) | (df4_poi['longitude'] == 0))]
df4_poi = df4_poi.dropna(subset=['longitude', 'latitude'])  # 결측값 제거
df4_poi["longitude"] = pd.to_numeric(df4_poi["longitude"], errors="coerce")
df4_poi["latitude"] = pd.to_numeric(df4_poi["latitude"], errors="coerce")

In [ ]:
region_total_mapping = {
    '서울특별시': ['서울']+sorted([
                        "종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", 
                        "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", 
                        "양천구", "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", 
                        "서초구", "강남구", "송파구", "강동구"
                        ], reverse=True),
    '세종특별자치시':["세종"],
    '부산광역시': ["부산"],
    '대구광역시': ['대구'],
    '인천광역시': ['인천'],
    '광주광역시': ["광주광역시"],
    '대전광역시': ["대전"],
    '울산광역시': ["울산"],
    '광주광역시': ["광주"],
    '경기도': ['경기', '수원시', '고양시', '화성시', '성남시', '용인시', '부천시', '안양시', '시흥시', '기타 경기도 지역'],
    '강원도': ['강원', '춘천시', '강릉시', '원주시', '기타 강원도 지역'],
    '경상남도': ['경남', '창원시', '김해시', '기타 경상남도 지역'],
    '경상북도': ['경북', '포항시', '경주시', '기타 경상북도 지역'],
    '전라남도': ['전남', '여수시', '순천시', '목포시', '기타 전라남도 지역'],
    '전라북도': ['전북', '전주시', '군산시', '익산시', '기타 전라북도 지역'],
    '충청남도': ['충남', '천안시', '아산시', '기타 충청남도 지역'],
    '충청북도': ['충북', '청주시', '충주시', '기타 충청북도 지역'],
    '제주특별자치도': ['제주', '서귀포']}

In [ ]:
# 매핑을 정규식으로 처리
for region, names in region_total_mapping.items():
    pattern = '|'.join([region]+names)  # 지역 리스트를 OR로 결합
    df1_greencar[region] = df1_greencar["address"].str.contains(pattern, case=False, na=False).astype(int)

# 결측값 처리 (필요한 경우)
df1_greencar["address"].fillna("", inplace=True)


In [ ]:
# 구별 처리
for i in region_total_mapping["서울특별시"][1:]:
    valid_columns = [col for col in region_total_mapping["서울특별시"][1:] if col in df1_greencar.columns]
    mask = (df1_greencar["서울특별시"]==1) & (df1_greencar[valid_columns].sum(axis=1) == 0)
    df1_greencar.loc[mask, i] = df1_greencar.loc[mask, "address"].str.contains(rf"{i}").astype(int)
    df1_greencar[i].fillna(0, inplace=True)
    df1_greencar[i] = df1_greencar[i].astype(int)

In [ ]:
# 기타{region}지역을 제외한 도별 주요 도시 처리
for region in list(region_total_mapping.keys())[8:-1]: #서울, 광역시, 제주 제외
    mask = (df1_greencar[f"{region}"]==1)
    for i in region_total_mapping[f"{region}"][1:-1]: #지역(대단위)와 기타{region}지역 제외
        valid_columns = [col for col in region_total_mapping[f"{region}"][1:-1] if col in df1_greencar.columns]
        mask = (df1_greencar[f"{region}"]==1) & (df1_greencar[valid_columns].sum(axis=1) == 0)
        j = i[:-1]
        df1_greencar.loc[mask, i] = df1_greencar.loc[mask, "address"].str.contains(rf"{j}").astype(int)
        # 나머지는 경기 여부를 0으로 설정
        df1_greencar[i].fillna(0, inplace=True)
        df1_greencar[i] = df1_greencar[i].astype(int)


In [ ]:
# 도별 주요 도시를 제외한 기타 지역 처리
for region in list(region_total_mapping.keys())[8:-1]: #서울, 광역시, 제주 제외 
    city_list = region_total_mapping[f"{region}"][1:-1]

    #조건 설정
    mask = (df1_greencar[f"{region}"]==1)&(df1_greencar[city_list].sum(axis=1) == 0)
    # 기타 지역 인코딩: 조건에 맞는 행을 모두 1로 설정
    df1_greencar.loc[mask, f"기타 {region} 지역"] = 1
    # 나머지는 경기 여부를 0으로 설정
    df1_greencar[f"기타 {region} 지역"].fillna(0, inplace=True)
    df1_greencar[f"기타 {region} 지역"] = df1_greencar[f"기타 {region} 지역"].astype(int)

In [ ]:
## 전처리 확인
total_count = 0
for region in list(region_total_mapping.keys())[:-1]: #제주 제외
    count = 0
    print("")
    for i in region_total_mapping[f"{region}"][1:]: 
        print(f"{i} :", len(df1_greencar[df1_greencar[i]==1]))
        count += len(df1_greencar[df1_greencar[i]==1])
    print("")
    print(f"{region} 총합 :", count)
    print(f"{region} whole_count", len(df1_greencar[df1_greencar[f"{region}"]==1]))
    
    total_count += len(df1_greencar[df1_greencar[f"{region}"]==1])

for region in  list(region_total_mapping.keys())[-1:]: 
    print(f"{region} whole_count", len(df1_greencar[df1_greencar[f"{region}"]==1]))
    total_count += len(df1_greencar[df1_greencar[f"{region}"]==1])

print("total_index_count", total_count)
total = len(df1_greencar)
print("데이터셋 전체:", total)

In [ ]:
# 4번째 열 이후의 열 선택
columns_to_check = df1_greencar.columns[4:]

# 1과 0이 아닌 값을 0으로 변경
df1_greencar[columns_to_check] = df1_greencar[columns_to_check].where(
    df1_greencar[columns_to_check].isin([0, 1]), 0
).astype(int)  # int로 변환

# 각 행의 합 계산
row_sums = df1_greencar[columns_to_check].sum(axis=1)

# row_sums가 0이거나 3인 행 삭제
filtered_df1_greencar = df1_greencar[(row_sums != 0) & (row_sums != 3)]

In [ ]:
SERVICE_ACCOUNT_FILE = "./key/api_key_carsharing.json"  # 키 json 파일
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
project_id = "carsharing-454809"
dataset_name = "carsharing"
client = bigquery.Client(credentials=credentials, project=project_id)

In [ ]:
to_gbq(df2_licenses, f"{dataset_name}.licenses", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.licenses")
to_gbq(filtered_df1_greencar, f"{dataset_name}.greencars", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.greencars")
to_gbq(df3_transport, f"{dataset_name}.transports", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.transports")
to_gbq(df4_poi, f"{dataset_name}.sites", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.sites")